In [1]:
import pickle
with open('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_diffcast128.pkl', 'rb') as f:
    results_diffcast_new = pickle.load(f)
with open('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_nowcasting128.pkl', 'rb') as f:
    results_nowcasting_new = pickle.load(f)
with open('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_vqbatched128_temp03topk70.pkl', 'rb') as f:
    results_vqbatched_new = pickle.load(f)
with open('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_vqbatched128_temp03topk70_s2s3.pkl', 'rb') as f:
    results_vqbatched_new_s2s3 = pickle.load(f)

#change names
results_diffcast_new = {'Diffcast+Phydnet': results_diffcast_new['diffcast_phydnet']}
results_nowcasting_new = {'NowcastingGPT': results_nowcasting_new['vqgan_GPT']}
results_vqbatched_new = {'BlockGPT': results_vqbatched_new['vqgan-GPT_batched']}
results_vqbatched_new_s2 = {'BlockGPT': results_vqbatched_new_s2s3['vqgan_s2_GPT_batched']}
results_vqbatched_new_s3 = {'BlockGPT': results_vqbatched_new_s2s3['vqgan_s3_GPT_batched']}

with open ('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_diffcast128_s2.pkl', 'rb') as f:
    results_diffcast_new_s2 = pickle.load(f)
results_diffcast_new_s2 = {'Diffcast+Phydnet': results_diffcast_new_s2['diffcast_phydnet']}

with open('/space2/vsarathchandra/blockGPT/Results/FinalPickledResults/roc_pr_results_new_nowcasting128_s3.pkl', 'rb') as f:
    results_nowcasting_new_s3 = pickle.load(f)

results_nowcasting_new_s3 = {'NowcastingGPT': results_nowcasting_new_s3['vqgan_s3_GPT']}

In [2]:
from evaluate_catchment import average_region_metrics, get_overall_metriscs
gt_thresholds=[1,2,8]

current_path is:  /space2/vsarathchandra/blockGPT


/space2/vsarathchandra/conda_envs/blockgpt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pysteps configuration file found at: /space2/vsarathchandra/conda_envs/blockgpt/lib/python3.9/site-packages/pysteps/pystepsrc



In [3]:
avg_metrics_vqbatched_s1 = average_region_metrics(results_vqbatched_new, gt_thresholds)
avg_metrics_vqbatched_s2 = average_region_metrics(results_vqbatched_new_s2, gt_thresholds)
avg_metrics_vqbatched_s3 = average_region_metrics(results_vqbatched_new_s3, gt_thresholds)
avg_merics_diffcasts1 = average_region_metrics(results_diffcast_new, gt_thresholds)
avg_merics_diffcasts2 = average_region_metrics(results_diffcast_new_s2, gt_thresholds)
avg_nowcasting = average_region_metrics(results_nowcasting_new, gt_thresholds)
avg_nowcasting_s3 = average_region_metrics(results_nowcasting_new_s3, gt_thresholds)


overall_metrics_vqbatched_s1 = get_overall_metriscs(results_vqbatched_new)
overall_metrics_vqbatched_s2 = get_overall_metriscs(results_vqbatched_new_s2)
overall_metrics_vqbatched_s3 = get_overall_metriscs(results_vqbatched_new_s3)
overall_metrics_diffcasts1 = get_overall_metriscs(results_diffcast_new)
overall_metrics_diffcasts2 = get_overall_metriscs(results_diffcast_new_s2)
overall_nowcasting = get_overall_metriscs(results_nowcasting_new)
overall_nowcasting_s3 = get_overall_metriscs(results_nowcasting_new_s3)

In [4]:
import numpy as np
from collections import defaultdict
import copy

def average_catchment_results(seed_dicts):
    """
    Averages per-time precision, recall, fpr, tpr, auc_roc, and auc_pr across multiple seeds.

    Args:
        seed_dicts (List[Dict]): List of result dictionaries (one per seed).

    Returns:
        Dict: Averaged dictionary structure across seeds.
    """
    # Use the first dict as a template
    template = seed_dicts[0]
    averaged_results = {}

    for model in template:
        averaged_results[model] = {}
        for threshold in template[model]:
            averaged_results[model][threshold] = {"per_time": []}
            
            # Assuming 9 time steps
            for t in range(len(template[model][threshold]["per_time"])):
                time_metrics = defaultdict(list)

                for seed in seed_dicts:
                    data = seed[model][threshold]["per_time"][t]
                    for key, value in data.items():
                        time_metrics[key].append(np.array(value))

                averaged_time_metrics = {
                    key: np.mean(np.stack(values, axis=0), axis=0)
                    for key, values in time_metrics.items()
                }

                averaged_results[model][threshold]["per_time"].append(averaged_time_metrics)
    
    return averaged_results

import numpy as np
from collections import defaultdict

def average_overall_results(seed_dicts):
    """
    Computes mean and std for per-time metrics across seeds.

    Args:
        seed_dicts (List[Dict]): List of result dictionaries (one per seed).

    Returns:
        Dict: Nested dict structure with averaged and std per-time metrics.
    """
    template = seed_dicts[0]
    averaged_results = {}

    for model in template:
        averaged_results[model] = {}
        for threshold in template[model]:
            averaged_results[model][threshold] = {"per_time": []}

            num_time_steps = len(template[model][threshold]["per_time"])
            for t in range(num_time_steps):
                time_metrics = defaultdict(list)

                for seed in seed_dicts:
                    data = seed[model][threshold]["per_time"][t]
                    for key, value in data.items():
                        time_metrics[key].append(np.array(value))

                averaged_time_metrics = {}
                for key, values in time_metrics.items():
                    stacked = np.stack(values, axis=0)  # (num_seeds, len_thresholds) or scalar
                    if stacked.ndim == 2:  # array metrics (precision, recall, etc.)
                        if key != 'confusion':
                            averaged_time_metrics[key] = np.mean(stacked, axis=0)
              
                    else:  # scalar metrics (auc_roc, auc_pr)
                        if key != 'confusion':
                            averaged_time_metrics[key] = float(np.mean(stacked))
                        

                averaged_results[model][threshold]["per_time"].append(averaged_time_metrics)

    return averaged_results


import numpy as np
from collections import defaultdict

def average_seed_results_with_std(seed_dicts):
    """
    Computes mean and std for per-time metrics across seeds.

    Args:
        seed_dicts (List[Dict]): List of result dictionaries (one per seed).

    Returns:
        Dict: Nested dict structure with averaged and std per-time metrics.
    """
    template = seed_dicts[0]
    averaged_results = {}

    for model in template:
        averaged_results[model] = {}
        for threshold in template[model]:
            averaged_results[model][threshold] = {"per_time": []}

            num_time_steps = len(template[model][threshold]["per_time"])
            for t in range(num_time_steps):
                time_metrics = defaultdict(list)

                for seed in seed_dicts:
                    data = seed[model][threshold]["per_time"][t]
                    for key, value in data.items():
                        time_metrics[key].append(np.array(value))

                averaged_time_metrics = {}
                for key, values in time_metrics.items():
                    stacked = np.stack(values, axis=0)  # (num_seeds, len_thresholds) or scalar
                    if stacked.ndim == 2:  # array metrics (precision, recall, etc.)
                        if key != 'confusion':
                            averaged_time_metrics[key + "_mean"] = np.mean(stacked, axis=0)
                            averaged_time_metrics[key + "_std"] = np.std(stacked, axis=0)
                    else:  # scalar metrics (auc_roc, auc_pr)
                        if key != 'confusion':
                            averaged_time_metrics[key + "_mean"] = float(np.mean(stacked))
                            averaged_time_metrics[key + "_std"] = float(np.std(stacked))

                averaged_results[model][threshold]["per_time"].append(averaged_time_metrics)

    return averaged_results


In [13]:
average_seed_results_vq = average_seed_results_with_std([avg_metrics_vqbatched_s1, avg_metrics_vqbatched_s2, avg_metrics_vqbatched_s3])
average_seed_results_diffcast = average_seed_results_with_std([avg_merics_diffcasts1, avg_merics_diffcasts2])
avergare_seed_results_nowcasting = average_seed_results_with_std([avg_nowcasting, avg_nowcasting_s3])
overall_seed_results_vq = average_seed_results_with_std([overall_metrics_vqbatched_s1, overall_metrics_vqbatched_s2, overall_metrics_vqbatched_s3])
overall_seed_results_diffcast = average_seed_results_with_std([overall_metrics_diffcasts1, overall_metrics_diffcasts2])
overall_seed_results_nowcasting = average_seed_results_with_std([overall_nowcasting, overall_nowcasting_s3])
combined_results_avg = {**average_seed_results_vq,**average_seed_results_diffcast, **avergare_seed_results_nowcasting}
combined_overall_results = {**overall_seed_results_vq, **overall_seed_results_diffcast, **overall_seed_results_nowcasting}

First, I plot the auc of roc and pr curves over time. In this plot, I show the standard deviation over the seeds as well. In the later plots, I show each pr and roc curve at every time instant. I could not figure out how to show standard deviation properly without messing up the plots

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import os
from datetime import datetime
from typing import Dict, List
import plotly.io as pio
import matplotlib.colors as mcolors

def to_rgba(color_str, alpha=0.2):
    """
    Convert a hex or rgb string to rgba string with specified alpha.
    """
    rgba = mcolors.to_rgba(color_str, alpha)
    return f'rgba({int(rgba[0]*255)}, {int(rgba[1]*255)}, {int(rgba[2]*255)}, {rgba[3]:.2f})'

def plot_auc_metrics_consistent_style(
    avg_results: Dict[str, dict],
    gt_thresholds: List[float],
    output_folder: str,
    context_length: int = 3,
    segment_length: int = 9,
    time_resolution: int = 30
):
    os.makedirs(output_folder, exist_ok=True)

    model_names = list(avg_results.keys())
    palette = px.colors.qualitative.Plotly
    color_map = {
        model: palette[i % len(palette)]
        for i, model in enumerate(model_names)
    }

    time_steps = list(range(context_length, segment_length))
    x_axis_labels = [f"{(i + 1) * time_resolution - 90}" for i in time_steps]

    for metric in ["auc_roc", "auc_pr"]:
        fig = make_subplots(
            rows=1,
            cols=len(gt_thresholds),
            shared_yaxes=False,
            subplot_titles=[f"GT ≥ {gt} mm" for gt in gt_thresholds]
        )

        for col, gt_thresh in enumerate(gt_thresholds, start=1):
            for model in model_names:
                if gt_thresh not in avg_results[model]:
                    continue

                per_time = avg_results[model][gt_thresh]["per_time"]
                mean_vals = [step[f"{metric}_mean"] for step in per_time][context_length:]
                std_vals = [step[f"{metric}_std"] for step in per_time][context_length:]

                lower_bound = [max(m - s, 0) for m, s in zip(mean_vals, std_vals)]
                upper_bound = [min(m + s, 1) for m, s in zip(mean_vals, std_vals)]

                # Plot shaded region
                fig.add_trace(
                    go.Scatter(
                        x=x_axis_labels + x_axis_labels[::-1],
                        y=upper_bound + lower_bound[::-1],
                        fill='toself',
                        fillcolor=to_rgba(color_map[model], alpha=0.2),
                        line=dict(color='rgba(255,255,255,0)'),
                        hoverinfo="skip",
                        showlegend=False,
                        legendgroup=model
                    ),
                    row=1, col=col
                )

                # Plot mean line
                fig.add_trace(
                    go.Scatter(
                        x=x_axis_labels,
                        y=mean_vals,
                        mode="lines+markers",
                        name=model,
                        line=dict(color=color_map[model]),
                        marker=dict(color=color_map[model]),
                        showlegend=(col == 1),
                        legendgroup=model
                    ),
                    row=1, col=col
                )

            fig.update_xaxes(title_text="Time (min)", row=1, col=col)
            fig.update_yaxes(
                title_text="AUC PR" if metric == "auc_pr" else "AUC ROC",
                row=1, col=col
            )

        fig.update_layout(
            title_text=f"AUC of {'PR' if metric == 'auc_pr' else 'ROC'} over Time by GT Threshold",
            title_x=0.5,
            height=400,
            width=400 * len(gt_thresholds),
            template="plotly_white",
            legend=dict(
                orientation="h",
                y=-0.2,
                x=0.5,
                xanchor="center"
            )
        )

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_path = os.path.join(output_folder, f"{metric}_over_time_{timestamp}.png")
        pio.write_image(fig, save_path, format="png", scale=3)
        print(f"Saved: {save_path}")


In [14]:
plot_auc_metrics_consistent_style(combined_results_avg, gt_thresholds, output_folder="Results/MetricsPlots/AUC_over_time_plots/KNMI30/Catchments")
plot_auc_metrics_consistent_style(combined_overall_results, gt_thresholds, output_folder="Results/MetricsPlots/AUC_over_time_plots/KNMI30/Overall")

Saved: Results/MetricsPlots/AUC_over_time_plots/KNMI30/Catchments/auc_roc_over_time_20250806_173558.png
Saved: Results/MetricsPlots/AUC_over_time_plots/KNMI30/Catchments/auc_pr_over_time_20250806_173558.png
Saved: Results/MetricsPlots/AUC_over_time_plots/KNMI30/Overall/auc_roc_over_time_20250806_173559.png
Saved: Results/MetricsPlots/AUC_over_time_plots/KNMI30/Overall/auc_pr_over_time_20250806_173559.png


This is the section which plots each roc and pr curve at every time instant

In [8]:
average_seed_results_vq = average_catchment_results([avg_metrics_vqbatched_s1, avg_metrics_vqbatched_s2, avg_metrics_vqbatched_s3])
average_seed_results_diffcast = average_catchment_results([avg_merics_diffcasts1, avg_merics_diffcasts2])
avergare_seed_results_nowcasting = average_catchment_results([avg_nowcasting, avg_nowcasting_s3])
overall_seed_results_vq = average_overall_results([overall_metrics_vqbatched_s1, overall_metrics_vqbatched_s2, overall_metrics_vqbatched_s3])
overall_seed_results_diffcast = average_overall_results([overall_metrics_diffcasts1, overall_metrics_diffcasts2])
overall_seed_results_nowcasting = average_overall_results([overall_nowcasting, overall_nowcasting_s3])
combined_results_avg = {**average_seed_results_vq,**average_seed_results_diffcast, **avergare_seed_results_nowcasting}
combined_overall_results = {**overall_seed_results_vq, **overall_seed_results_diffcast, **overall_seed_results_nowcasting}

In [9]:

def plot_time_sliced_curves_for_models_plotly_png(
    avg_results: Dict[str, dict],
    gt_thresholds: List[float],
    metrics: List[str] = ["roc", "pr"],
    prefix: str = "",
    save_dir: str = "./plots"
):
    import os
    from datetime import datetime
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.express as px
    import plotly.io as pio

    os.makedirs(save_dir, exist_ok=True)

    model_names = list(avg_results.keys())
    palette = px.colors.qualitative.Plotly
    color_map = {model: palette[i % len(palette)] for i, model in enumerate(model_names)}

    for metric in metrics:
        assert metric in ["roc", "pr"], "Metric must be 'roc' or 'pr'"
        metric_label = "ROC" if metric == "roc" else "PR"

        for gt_thresh in gt_thresholds:
            fig = make_subplots(
                rows=2,
                cols=3,
                subplot_titles=[f"{prefix}{metric_label} | Time = {30 * (t - 2)} min" for t in range(3, 9)],
                shared_xaxes=False,
                shared_yaxes=False,
                horizontal_spacing=0.1,
                vertical_spacing=0.27  # increased spacing between top and bottom rows
            )

            for i, t in enumerate(range(3, 9)):
                row, col = divmod(i, 3)
                row += 1
                col += 1
                subplot_index = i + 1

                x_label = "False Positive Rate" if metric == "roc" else "Recall"
                y_label = "True Positive Rate" if metric == "roc" else "Precision"

                x_min, x_max = float("inf"), float("-inf")
                y_min, y_max = float("inf"), float("-inf")
                auc_texts = []

                for model in model_names:
                    per_time = avg_results[model].get(gt_thresh, {}).get("per_time", [])
                    if t >= len(per_time):
                        continue

                    step = per_time[t]
                    auc_val = step.get("auc_roc" if metric == "roc" else "auc_pr", None)
                    if auc_val is None:
                        continue

                    x_vals = step.get("fpr" if metric == "roc" else "recall", [])
                    y_vals = step.get("tpr" if metric == "roc" else "precision", [])

                    if len(x_vals) == 0 or len(y_vals) == 0:
                        continue

                    x_min = min(x_min, min(x_vals))
                    x_max = max(x_max, max(x_vals))
                    y_min = min(y_min, min(y_vals))
                    y_max = max(y_max, max(y_vals))

                    fig.add_trace(
                        go.Scatter(
                            x=x_vals,
                            y=y_vals,
                            mode="lines",
                            name=model,
                            line=dict(color=color_map[model]),
                            legendgroup=model,
                            showlegend=(i == 0)
                        ),
                        row=row,
                        col=col
                    )

                    auc_texts.append((model, auc_val))

                x_margin = 0.05 * (x_max - x_min) if x_max > x_min else 0.01
                y_margin = 0.05 * (y_max - y_min) if y_max > y_min else 0.01
                fig.update_xaxes(title_text=x_label, range=[x_min - x_margin, x_max + x_margin], row=row, col=col)
                fig.update_yaxes(title_text=y_label, range=[y_min - y_margin, y_max + y_margin], row=row, col=col)

                # Safe AUC text positioning below x-axis labels
                base_y = 0.55 if row == 1 else -0.08  # lower bottom row further
                for j, (model, auc_val) in enumerate(auc_texts):
                    fig.add_annotation(
                        text=f"{model}: AUC={auc_val:.3f}",
                        xref=f"x{subplot_index}",
                        yref="paper",
                        x=(x_min + x_max) / 2,
                        y=base_y - j * 0.04,
                        xanchor="center",
                        yanchor="top",
                        showarrow=False,
                        font=dict(size=10, color=color_map[model])
                    )

            fig.update_layout(
                height=950,  # increased from 850
                width=1100,
                title_text=f"{metric_label} Curves over Time (GT ≥ {gt_thresh} mm)",
                title_x=0.5,
                template="plotly_white",
                legend=dict(
                    orientation="h",
                    y=-0.35,
                    x=0.5,
                    xanchor="center"
                )
            )

            filename = f"{metric}_gt_{gt_thresh}mm_plotly.png"
            filepath = os.path.join(save_dir, filename)
            pio.write_image(fig, filepath, format="png", scale=3)
            print(f"Saved: {filepath}")


In [11]:
plot_time_sliced_curves_for_models_plotly_png(combined_results_avg, gt_thresholds, prefix="Aggregated Catchment ", save_dir="Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments")
plot_time_sliced_curves_for_models_plotly_png(combined_overall_results, gt_thresholds, prefix="Aggregated Catchment ", save_dir="Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall")

Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/roc_gt_1mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/roc_gt_2mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/roc_gt_8mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/pr_gt_1mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/pr_gt_2mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Catchments/pr_gt_8mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/roc_gt_1mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/roc_gt_2mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/roc_gt_8mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/pr_gt_1mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/pr_gt_2mm_plotly.png
Saved: Results/MetricsPlots/ROC_PR_plots/KNMI30/Overall/pr_gt_8mm_plotly.png
